In [ ]:
import sys
# import modules in other directories
sys.path.append('../Utils/')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import copy
import importlib
import get_lens1_rcp85 as lens1
import sst_biases_2018pub as sstbias

import trax_util as trx
import ibtracs as IBT
import ts_w_trx_joint as ttJ
import PlotUtil as Pu

importlib.reload( Pu )
importlib.reload( sstbias )
importlib.reload( lens1 )
importlib.reload( trx )
importlib.reload( IBT )
importlib.reload( ttJ )


In [ ]:
#return BACE_stack,ts_stack,BACE_pd_stack,had_stack,landf,lat,lon
BACE_stack, ts_stack, BACE_pd_stack, had_stack, landf, lat, lon = ttJ.trx_w_ts()

### Obs
ibx=IBT.readtrx()
BACE_ibx,YrI,MonI =trx.basinACEmonth(wind=ibx.wind,basin=ibx.basin,year=ibx.year,month=ibx.month,power_wind=1)


## AT this point you've read in the LENS TS data for 2070-2099 with pre-pended TS from an ealrier CESM1 coupled run
## Also have IBtrACS and HADLEY SSTs

In [ ]:
####
# Deep Copies
####
ts_stack_O = copy.deepcopy( ts_stack )
had_stack_O = copy.deepcopy( had_stack )

In [ ]:
#### 
# Deseasonalize ...
###
nens,nYr,nMo,ny,nx = np.shape(ts_stack)
ts_stack_A = np.zeros( (nens,nMo,ny,nx ) )
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        ts_stack_A[:,im,:,:] = ts_stack_A[:,im,:,:] + ts_stack[:,iy,im,:,:] 
ts_stack_A = ts_stack_A / nYr
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        ts_stack[:,iy,im,:,:] = ts_stack[:,iy,im,:,:] - ts_stack_A[:,im,:,:]


nens,nYr,nMo,ny,nx = np.shape(had_stack)
had_stack_A = np.zeros( (nens,nMo,ny,nx ) )
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        had_stack_A[:,im,:,:] = had_stack_A[:,im,:,:] + had_stack[:,iy,im,:,:] 
had_stack_A = had_stack_A / nYr
for im in np.arange( 12 ):
    for iy in np.arange( nYr ):
        had_stack[:,iy,im,:,:] = had_stack[:,iy,im,:,:] - had_stack_A[:,im,:,:]




In [ ]:
plt.plot( ts_stack[0,:,:,100,92].flatten() )

In [ ]:
#         -1.       0.        1.         2            3             4              5            6 
bnames=['None','N Atlantic', 'S Atlantic' , 'NW Pacific', 'NE Pacific', 'SW Pacific','N Indian','S Indian']


In [ ]:
######
# Set N Atl basin
######
b=1

###########
# Set months for anlaysis
# e.g. mon0,mon1=6,9
# means we look at JJAS
############
mon0,mon1 = 5,11 # MJJASON
#mon0,mon1 = 6,9 # JJAS
#mon0,mon1 = 8,10 # ASO
#mon0,mon1=9,9
month_idxs =np.array( [5,6,7,8,9,10,11] )-1
print(month_idxs)


In [ ]:
######
# Set S Inidan basin
######
b=7

###########
# Set months for anlaysis
# e.g. mon0,mon1=6,9
# means we look at JJAS
############
mon0,mon1 = 1,12 # MJJASON
#mon0,mon1 = 5,11 # MJJASON
#mon0,mon1 = 6,9 # JJAS
#mon0,mon1 = 8,10 # ASO
#mon0,mon1=9,9

#mon0-1:mon1=np.array( [1,2,3,4,5,10,11,12] )-1


In [ ]:
print(np.shape(BACE_stack[:,:,:,month_idxs]))
print(np.shape(ts_stack[:,:,month_idxs,:,:]))
nens,nYr,nMo,ny,nx = np.shape(ts_stack[:,:,month_idxs,:,:])
ts_stackR = np.reshape( ts_stack[:,:,month_idxs,:,:]   , (nens*nYr*nMo, ny,nx) )
tmp = BACE_stack[:,:,:,month_idxs]
BACE_stackR = np.reshape( tmp[:,b,:,:]   , (nens*nYr*nMo ) )

print(np.shape(BACE_pd_stack[:,:,:,month_idxs]))
print(np.shape(had_stack[:,:,month_idxs,:,:]))
nens,nYr,nMo,ny,nx = np.shape(had_stack[:,:,month_idxs,:,:])
had_stackR = np.reshape( had_stack[:,:,month_idxs,:,:]   , (nens*nYr*nMo, ny,nx) )
tmp = BACE_pd_stack[:,:,:,month_idxs]
BACE_pd_stackR = np.reshape( tmp[:,b,:,:]   , (nens*nYr*nMo ) )

#BACE_pd_stackR_2 = np.reshape( BACE_pd_stack[:,b,:,month_idxs]   , (nens*nYr*nMo ) )

In [ ]:
hho,xe=np.histogram( BACE_stackR , bins=18, range=(1,180) )
hhp,xe=np.histogram( BACE_pd_stackR , bins=18, range=(1,180) )

#hho=hho/np.size(BACE_stackR)
#hhp=hhp/np.size(BACE_pd_stackR)

plt.semilogy( xe[0:-1],hho,'o-' )
plt.semilogy( xe[0:-1],hhp,'o-',ms=3 )
#plt.ylim(.5,220)

In [ ]:
hho,xe=np.histogram( BACE_stackR , bins=18, range=(1,180) )
hhp,xe=np.histogram( BACE_pd_stackR , bins=18, range=(1,180) )

#hho=hho/np.size(BACE_stackR)
#hhp=hhp/np.size(BACE_pd_stackR)
fig = plt.figure(figsize=(10, 5))
Axes1 = Pu.axes_def(n=1,nxplo=2,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
h1=ax1.hist(BACE_stackR , bins=18, range=(1,180),density=True)
Axes1 = Pu.axes_def(n=2,nxplo=2,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
h2=ax1.hist(BACE_pd_stackR , bins=18, range=(1,180),density=True)
#plt.semilogy( xe[0:-1],hhp,'o-',ms=3 )
#plt.ylim(.5,220)

In [ ]:
NEPac4tup=(85,100,190,220)
NAtl4tup=(110,120,240,270)


In [ ]:
############
# Plot SSTs associated high and low ACE in selected basin
############

c1=np.asarray( (NEPac4tup) )
c2=np.asarray( (NAtl4tup)  )


h1,xACE=np.histogram(BACE_pd_stackR , bins=1800, range=(0,180)  ,density=True)
#print( np.sum( xACE[1:]*h1 * (xACE[1]-xACE[0])) ,xACE[0:2] )
meanMonACE_PD  =  np.sum( xACE[1:]*h1 * (xACE[1]-xACE[0]))
h1,xACE=np.histogram(BACE_stackR , bins=1800, range=(0,180)  ,density=True)
#print( np.sum( xACE[1:]*h1 * (xACE[1]-xACE[0])) ,xACE[0:2] )
meanMonACE_RCP =  np.sum( xACE[1:]*h1 * (xACE[1]-xACE[0]))

print( meanMonACE_PD , meanMonACE_RCP )


levels=2*np.linspace(-1,1,num=21)

fig = plt.figure(figsize=(20, 15))

c1=np.asarray( (NEPac4tup) )
c2=np.asarray( (NAtl4tup)  )


nxplo=2
nyplo=3
n=0




n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ,xpan=0.71, ycor=0.89,ypan=0.7) 
ax1 = fig.add_axes( Axes )
h1=ax1.hist(BACE_stackR , bins=18*2, range=(0,180),density=True)
ax1.set_xlabel( "Monthly Mean N Atl ACE",fontsize=16)
ax1.set_ylabel( "Probability",fontsize=16)
ax1.set_title( "Future SSTs 2020-2100", fontsize=20)
ax1.set_ylim(3e-4,0.15)
ax1.set_yscale('log')
ax1.axvline(meanMonACE_RCP, color='r', linestyle='dashed', linewidth=1)
plt.text(meanMonACE_RCP + 3, plt.ylim()[1] * 0.25, f'Mean: {meanMonACE_RCP:.1f}', color='r',fontsize=16)
n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ,xpan=0.71 , ycor=0.89, ypan=0.7)
ax1 = fig.add_axes( Axes )
h2=ax1.hist(BACE_pd_stackR , bins=18*2, range=(0,180),density=True)
ax1.set_xlabel( "Monthly Mean N Atl ACE",fontsize=16)
ax1.set_ylabel( "Probability",fontsize=16)
ax1.set_title( "Present Day ensemble 1980-2012", fontsize=20)
ax1.set_ylim(3e-4,0.15)
ax1.set_yscale('log')
ax1.axvline(meanMonACE_PD, color='r', linestyle='dashed', linewidth=1)
plt.text(meanMonACE_PD + 3, plt.ylim()[1] * 0.25, f'Mean: {meanMonACE_PD:.1f}', color='r',fontsize=16)


levels=0.25*np.linspace(-1,1,num=21)


ACEo = meanMonACE_RCP
zoo = np.where( BACE_stackR > ACEo)
ts0 = np.average( ts_stackR , axis=0 )
ts1 = np.average( ts_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( dts, levels=levels,cmap='bwr'  )
bxw = ax1.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax1.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb1 = plt.colorbar(co1)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly N Atlantic ACE > {ACEo:.1f}",fontsize=16)
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_stackR ) )

ACEo = meanMonACE_PD
mince = str( ACEo)
zoo = np.where( BACE_pd_stackR > ACEo )
ts0 = np.average( had_stackR , axis=0 )
ts1 = np.average( had_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes2 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax2 = fig.add_axes( Axes2 )
co2 = ax2.contourf( dts, levels=levels,cmap='bwr' )
bxw = ax2.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax2.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb2 = plt.colorbar(co2)
ax2.set_xticks([])
ax2.set_xlabel('')
ax2.set_yticks([])
ax2.set_ylabel('')
#ax2.set_title("Monthly N Atlantic ACE > Mean",fontsize=16)
ax2.set_title(f"Monthly N Atlantic ACE > {ACEo:.1f}",fontsize=16)
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_pd_stackR ) )



levels=0.25*np.linspace(-1,1,num=21)

ACEo = meanMonACE_RCP
zoo = np.where( BACE_stackR < ACEo)
ts0 = np.average( ts_stackR , axis=0 )
ts1 = np.average( ts_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( dts, levels=levels,cmap='bwr'  )
bxw = ax1.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax1.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb1 = plt.colorbar(co1)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
#ax1.set_title("Monthly N Atlantic ACE < Mean",fontsize=16)
ax1.set_title(f"Monthly N Atlantic ACE < {ACEo:.1f}",fontsize=16)
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_stackR ) )

ACEo = meanMonACE_PD
zoo = np.where( BACE_pd_stackR < ACEo )
ts0 = np.average( had_stackR , axis=0 )
ts1 = np.average( had_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes2 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax2 = fig.add_axes( Axes2 )
co2 = ax2.contourf( dts, levels=levels,cmap='bwr' )
bxw = ax2.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax2.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb2 = plt.colorbar(co2)
ax2.set_xticks([])
ax2.set_xlabel('')
ax2.set_yticks([])
ax2.set_ylabel('')
#ax2.set_title("Monthly N Atlantic ACE < Mean",fontsize=16)
ax2.set_title(f"Monthly N Atlantic ACE < {ACEo:.1f}",fontsize=16)
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_pd_stackR ) )


In [ ]:
levels=0.5*np.linspace(-1,1,num=21)

fig = plt.figure(figsize=(20, 5))

c1=np.asarray( (NEPac4tup) )
c2=np.asarray( (NAtl4tup)  )

ACEo = 17.

zoo = np.where( BACE_stackR > ACEo)
ts0 = np.average( ts_stackR , axis=0 )
ts1 = np.average( ts_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

Axes1 = Pu.axes_def(n=1,nxplo=2,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( dts, levels=levels,cmap='bwr'  )
bxw = ax1.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax1.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb1 = plt.colorbar(co1)
print( np.size(zoo[0]) / np.size( BACE_stackR ) )

ACEo = 30.
zoo = np.where( BACE_pd_stackR > ACEo )
ts0 = np.average( had_stackR , axis=0 )
ts1 = np.average( had_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

Axes2 = Pu.axes_def(n=2,nxplo=2,nyplo=1 ) 
ax2 = fig.add_axes( Axes2 )
co2 = ax2.contourf( dts, levels=levels,cmap='bwr' )
bxw = ax2.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax2.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb2 = plt.colorbar(co2)
print( np.size(zoo[0]) / np.size( BACE_pd_stackR ) )


In [ ]:
levels=0.2*np.linspace(-1,1,num=21)

fig = plt.figure(figsize=(20, 5))

c1=np.asarray( (NEPac4tup) )
c2=np.asarray( (NAtl4tup)  )

ACEo = 13. #mean

zoo = np.where( BACE_stackR < ACEo)
ts0 = np.average( ts_stackR , axis=0 )
ts1 = np.average( ts_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

Axes1 = Pu.axes_def(n=1,nxplo=2,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( dts, levels=levels,cmap='bwr'  )
bxw = ax1.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax1.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb1 = plt.colorbar(co1)
print( np.size(zoo[0]) / np.size( BACE_stackR ) )

ACEo = 19. #mean
zoo = np.where( BACE_pd_stackR < ACEo )
ts0 = np.average( had_stackR , axis=0 )
ts1 = np.average( had_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

Axes2 = Pu.axes_def(n=2,nxplo=2,nyplo=1 ) 
ax2 = fig.add_axes( Axes2 )
co2 = ax2.contourf( dts, levels=levels,cmap='bwr' )
bxw = ax2.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax2.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb2 = plt.colorbar(co2)
print( np.size(zoo[0]) / np.size( BACE_pd_stackR ) )


In [ ]:
levels=2*np.linspace(-1,1,num=21)

fig = plt.figure(figsize=(20, 5))

c1=np.asarray( (NEPac4tup) )
c2=np.asarray( (NAtl4tup)  )

ACEo = 88.

zoo = np.where( BACE_stackR > ACEo)
ts0 = np.average( ts_stackR , axis=0 )
ts1 = np.average( ts_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

Axes1 = Pu.axes_def(n=1,nxplo=2,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( dts, levels=levels,cmap='bwr'  )
bxw = ax1.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax1.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb1 = plt.colorbar(co1)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_stackR ) )

ACEo = 105.
zoo = np.where( BACE_pd_stackR > ACEo )
ts0 = np.average( had_stackR , axis=0 )
ts1 = np.average( had_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

Axes2 = Pu.axes_def(n=2,nxplo=2,nyplo=1 ) 
ax2 = fig.add_axes( Axes2 )
co2 = ax2.contourf( dts, levels=levels,cmap='bwr' )
bxw = ax2.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax2.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb2 = plt.colorbar(co2)
ax2.set_xticks([])
ax2.set_xlabel('')
ax2.set_yticks([])
ax2.set_ylabel('')
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_pd_stackR ) )


In [ ]:
print(np.average(BACE_pd_stackR))
print(np.average(BACE_stackR))

In [ ]:
levels=2*np.linspace(-1,1,num=21)

fig = plt.figure(figsize=(20, 15))

c1=np.asarray( (NEPac4tup) )
c2=np.asarray( (NAtl4tup)  )


nxplo=2
nyplo=3
n=0




n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ,xpan=0.71, ycor=0.89,ypan=0.7) 
ax1 = fig.add_axes( Axes )
h1=ax1.hist(BACE_stackR , bins=18, range=(1,180),density=True)
ax1.set_xlabel( "Monthly Mean N Atl ACE",fontsize=16)
ax1.set_ylabel( "Probability",fontsize=16)
ax1.set_title( "Future SSTs 2020-2100", fontsize=20)
n=n+1
Axes = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ,xpan=0.71 , ycor=0.89, ypan=0.7)
ax1 = fig.add_axes( Axes )
h2=ax1.hist(BACE_pd_stackR , bins=18, range=(1,180),density=True)
ax1.set_xlabel( "Monthly Mean N Atl ACE",fontsize=16)
ax1.set_ylabel( "Probability",fontsize=16)
ax1.set_title( "Present Day ensemble 1980-2012", fontsize=20)







ACEo = 17.
levels=0.5*np.linspace(-1,1,num=21)

zoo = np.where( BACE_stackR > ACEo)
ts0 = np.average( ts_stackR , axis=0 )
ts1 = np.average( ts_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( dts, levels=levels,cmap='bwr'  )
bxw = ax1.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax1.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb1 = plt.colorbar(co1)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title("25$^{th}$ %-ile of Monthly N Atlantic ACE",fontsize=16)
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_stackR ) )

ACEo = 30.
zoo = np.where( BACE_pd_stackR > ACEo )
ts0 = np.average( had_stackR , axis=0 )
ts1 = np.average( had_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes2 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax2 = fig.add_axes( Axes2 )
co2 = ax2.contourf( dts, levels=levels,cmap='bwr' )
bxw = ax2.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax2.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb2 = plt.colorbar(co2)
ax2.set_xticks([])
ax2.set_xlabel('')
ax2.set_yticks([])
ax2.set_ylabel('')
ax2.set_title("25$^{th}$ %-ile of Monthly N Atlantic ACE",fontsize=16)
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_pd_stackR ) )




ACEo = 88.
levels=2*np.linspace(-1,1,num=21)

zoo = np.where( BACE_stackR > ACEo)
ts0 = np.average( ts_stackR , axis=0 )
ts1 = np.average( ts_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( dts, levels=levels,cmap='bwr'  )
bxw = ax1.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax1.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb1 = plt.colorbar(co1)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title("2$^{nd}$ %-ile of Monthly N Atlantic ACE",fontsize=16)
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_stackR ) )

ACEo = 105.
zoo = np.where( BACE_pd_stackR > ACEo )
ts0 = np.average( had_stackR , axis=0 )
ts1 = np.average( had_stackR[zoo[0],:,:] , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes2 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax2 = fig.add_axes( Axes2 )
co2 = ax2.contourf( dts, levels=levels,cmap='bwr' )
bxw = ax2.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax2.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb2 = plt.colorbar(co2)
ax2.set_xticks([])
ax2.set_xlabel('')
ax2.set_yticks([])
ax2.set_ylabel('')
ax2.set_title("2$^{nd}$ %-ile of Monthly N Atlantic ACE",fontsize=16)
print( np.size(zoo[0]), np.size(zoo[0]) / np.size( BACE_pd_stackR ) )


In [ ]:
####
meanMonACE_PD  =  np.average( BACE_pd_stackR)
meanMonACE_RCP =  np.average( BACE_stackR)

print( meanMonACE_PD , meanMonACE_RCP )


In [ ]:
had_stack.shape

In [ ]:
####
ts1=np.average( np.average( np.average( ts_stack_O , axis=0),axis=0),axis=0) 

ts0=np.average( np.average( np.average( had_stack_O , axis=0),axis=0),axis=0) 


In [ ]:
levels=2*np.linspace(-1,1,num=21)

fig = plt.figure(figsize=(10,7))

c1=np.asarray( (NEPac4tup) )
c2=np.asarray( (NAtl4tup)  )


nxplo=1
nyplo=1
n=0


levels=np.linspace(-10,10,num=21)


ACEo = meanMonACE_RCP
zoo = np.where( BACE_stackR > ACEo)
#ts1 = np.average( ts_stackR , axis=0 )
#ts0 = np.average( had_stackR , axis=0 )
dts = np.where( landf<.001, ts1 - ts0, np.nan ) 

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf(dts, levels=levels,cmap='bwr'  )
bxw = ax1.plot(  [ c1[2],c1[2],c1[3],c1[3],c1[2] ] ,  [c1[0],c1[1],c1[1],c1[0],c1[0]] ,color='black')
bxe = ax1.plot(  [ c2[2],c2[2],c2[3],c2[3],c2[2] ] ,  [c2[0],c2[1],c2[1],c2[0],c2[0]] ,color='black')
cb1 = plt.colorbar(co1)
ax1.set_xticks([])
ax1.set_xlabel('')
ax1.set_yticks([])
ax1.set_ylabel('')
ax1.set_title(f"Monthly N Atlantic ACE > {ACEo:.1f}",fontsize=16)


In [ ]:
# Create time series
ys,yn,xw,xe = NEPac4tup #80,110,150,200
ts_NEPacX = np.average( np.average( ts_stack[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )
print( f"NE Pac Region limits. Lon: {lon[xw]-360:.2f}, {lon[xe]-360:.2f} Lat: {lat[ys]:.2f}, {lat[yn]:.2f} ")


ys,yn,xw,xe = NAtl4tup #100,130,225,260
ts_NAtlX = np.average( np.average( ts_stack[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )
print( f"N Atl Region limits. Lon: {lon[xw]-360:.2f}, {lon[xe]-360:.2f} Lat: {lat[ys]:.2f}, {lat[yn]:.2f} ")


ts_Diff = 1.*ts_NEPacX - ts_NAtlX


In [ ]:
# Create time series
ys,yn,xw,xe = NEPac4tup
had_NEPac = np.average( np.average( had_stack[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )
print( f"NE Pac Region limits. Lon: {lon[xw]:.2f}, {lon[xe]:.2f} Lat: {lat[ys]:.2f}, {lat[yn]:.2f} ")


ys,yn,xw,xe = NAtl4tup #100,130,225,260
had_NAtl = np.average( np.average( had_stack[ :,:,:,ys:yn,xw:xe] , axis=4) , axis=3 )
print( f"N Atl Region limits. Lon: {lon[xw]:.2f}, {lon[xe]:.2f} Lat: {lat[ys]:.2f}, {lat[yn]:.2f} ")

had_Diff = 1.*had_NEPac - had_NAtl
print(np.shape(had_Diff))


In [ ]:
#####
# Prep for scatter plot diagnostsics
#####
monthly_style=True
if (monthly_style==True):
    x=ts_Diff[:,:,mon0-1:mon1].flatten()
    y=BACE_stack[:,b,:,mon0-1:mon1].flatten()
    c=ts_NAtlX[:,:,mon0-1:mon1].flatten()


    x0=had_Diff[:,:,mon0-1:mon1].flatten()
    y0=BACE_pd_stack[:,b,:,mon0-1:mon1].flatten()
    c0=had_NAtl[:,:,mon0-1:mon1].flatten()


    xo=had_Diff[0,:,mon0-1:mon1].flatten()
    yo=BACE_ibx[b,0:33,mon0-1:mon1].flatten()
    pdf_range = [(-3,4),(0.,170.)] 
    #pdf_range = [(-2,2),(0.,170.)] 
    
else:
    x=np.average( ts_Diff[:,:,mon0-1:mon1],axis=2).flatten()
    y=np.average( BACE_stack[:,b,:,mon0-1:mon1],axis=2).flatten()
    c=np.average( ts_NAtlX[:,:,mon0-1:mon1],axis=2).flatten()


    x0=np.average( had_Diff[:,:,mon0-1:mon1],axis=2).flatten()
    y0=np.average( BACE_pd_stack[:,b,:,mon0-1:mon1],axis=2).flatten()
    c0=np.average( had_NAtl[:,:,mon0-1:mon1],axis=2).flatten()


    xo=np.average( had_Diff[0,:,mon0-1:mon1],axis=1).flatten()
    yo=np.average( BACE_ibx[b,0:33,mon0-1:mon1],axis=1).flatten()
    pdf_range = [(-2,2),(0.,50.)]

In [ ]:
nbins=[15,10]
H0,X0,Y0=np.histogram2d(x0, y0, bins=nbins   ,range=pdf_range )
print(np.sum(H0))
print(np.shape(x0))
print(np.size(x0))
H0  =  H0/np.size(x0) #np.sum(H0)

Hr,Xr,Yr=np.histogram2d(x, y, bins=nbins   ,range=pdf_range )
print(np.sum(Hr))
print(np.shape(x))
Hr  =  Hr/np.size(x) #np.sum(Hr)

Ho,Xo,Yo=np.histogram2d(xo, yo, bins=nbins   ,range=pdf_range )
print(np.sum(Ho))
print(np.shape(xo))
Ho  =  Ho/np.size(xo) #np.sum(Ho)

XX=(X0[1:]+X0[:-1])/2
YY=(Y0[1:]+Y0[:-1])/2

#XX=X0[:-1]
YY=Y0[:-1]

H0_y = np.sum( H0.T , axis =0 )
H0_x = np.sum( H0.T , axis =1 )

Ho_y = np.sum( Ho.T , axis =0 )
Ho_x = np.sum( Ho.T , axis =1 )

Hr_y = np.sum( Hr.T , axis =0 )
Hr_x = np.sum( Hr.T , axis =1 )



In [ ]:
#lev=np.array( [0.001, 0.002,0.005,0.01,0.02,0.03,0.05,0.07,0.1] )
lev=np.array( [0.002,0.005,0.01,0.02,0.03,0.05,0.07,0.1] )
print(lev)

cmap=Pu.step_cmap(levels=lev,cmapN='gist_ncar')
nxplo,nyplo=3,1

fig = plt.figure(figsize=(20, 5))

xr0,xr1= pdf_range[0][0],pdf_range[0][1] # -6,2 #-4,2.7 #-3,3.7
b=1
#plt.plot( BACE_sst1[b,:,:].flatten() )

Axes0 = Pu.axes_def(n=1,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
hoo=ax1.contour(XX,YY, Ho.T,levels=lev,cmap=cmap,linewidths=4)
ska=ax1.scatter( xo, yo ,s=20, color='black')
n,m=nbins[0]-3,nbins[1]-2
box=ax1.plot( [ XX[n],XX[n+1],XX[n+1],XX[n],XX[n] ],[ YY[m],YY[m],YY[m+1],YY[m+1],YY[m] ],color='black' ) 
#foo=ax1.plot( [-4,3],[160,0] ,color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(pdf_range[1] ) #0,160)


###################
Axes0 = Pu.axes_def(n=2,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
hoo=ax1.contour(XX,YY, H0.T,levels=lev,cmap=cmap,linewidths=4)
ska=ax1.scatter( x0, y0 ,s=20, color='black')
#foo=ax1.plot( [-4,3],[160,0] ,color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(pdf_range[1] ) #0,160)


Axes0 = Pu.axes_def(n=3,nxplo=nxplo,nyplo=nyplo) 
ax1 = fig.add_axes( Axes0 )
hoo=ax1.contour(XX,YY, Hr.T,levels=lev,cmap=cmap,linewidths=4)
#foo=ax1.plot( [-4,3],[160,0] ,color='black')
ska=ax1.scatter( x,  y ,s=20, color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(pdf_range[1] ) #0,160)


cax=fig.add_axes( [ .2,-.2,.5,.02 ] )
fig.colorbar( hoo, cax=cax, orientation='horizontal' )
#plt.contourf(XX,YY, H0.T,levels=lev,cmap=cmap)
#plt.colorbar()

In [ ]:
#STOP

In [ ]:
#STOP

In [ ]:
x=ts_Diff[:,:,mon0-1:mon1].flatten()
y=BACE_stack[:,b,:,mon0-1:mon1].flatten()
plt.scatter( x,y,s=10 )
x0=had_Diff[:,:,mon0-1:mon1].flatten()
y0=BACE_pd_stack[:,b,:,mon0-1:mon1].flatten()
plt.scatter( x0,y0,s=10 )


In [ ]:
lev=np.array( [1e-5,0.0005,0.001,0.002,0.005,0.01,0.02,0.05, 0.1 ]) #, 0.2 ] )

lev=np.logspace( -3,-1, num=11)

lev=np.array( [0.001, 0.002,0.005,0.01,0.02,0.03,0.05,0.07,0.1] )
print(lev)

cmap=Pu.step_cmap(levels=lev,cmapN='gist_ncar')
nxplo,nyplo=3,1

fig = plt.figure(figsize=(20, 5))

xr0,xr1=-3,3.7
b=1
#plt.plot( BACE_sst1[b,:,:].flatten() )

Axes0,Axes1,Axes2 = Pu.axes_def(n=1,nxplo=nxplo,nyplo=nyplo,top_panel=.1,left_panel=.04 )
ax1 = fig.add_axes( Axes0 )
hoo=ax1.contourf(XX,YY, Ho.T,levels=lev,cmap=cmap)
foo=ax1.plot( [-3,4],[160,0] ,color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(0,160)

ax2 = fig.add_axes( Axes1 )
ax2.plot( XX, np.sum( Ho.T , axis=0) )
ax2.set_xlim(xr0,xr1)
#ax1.set_ylim(0,160)


ax3 = fig.add_axes( Axes2 )
ax3.plot( np.sum( Ho.T , axis=1),YY )
ax3.set_xlim(0.01,1)
ax3.set_xscale('log')
ax3.set_ylim(0,160)


###################
Axes0,Axes1,Axes2 = Pu.axes_def(n=2,nxplo=nxplo,nyplo=nyplo,top_panel=.1,left_panel=.04 )
ax1 = fig.add_axes( Axes0 )
hoo=ax1.contourf(XX,YY, H0.T,levels=lev,cmap=cmap)
foo=ax1.plot( [-3,4],[160,0] ,color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(0,160)

ax2 = fig.add_axes( Axes1 )
ax2.plot( XX, np.sum( H0.T , axis=0) )
ax2.set_xlim(xr0,xr1)
#ax1.set_ylim(0,160)


ax3 = fig.add_axes( Axes2 )
ax3.plot( np.sum( H0.T , axis=1),YY )
#ax1.set_xlim(-3,3)
ax3.set_xlim(0.01,1)
ax3.set_xscale('log')
ax3.set_ylim(0,160)


Axes0,Axes1,Axes2 = Pu.axes_def(n=3,nxplo=nxplo,nyplo=nyplo,top_panel=.1,left_panel=.04 )
ax1 = fig.add_axes( Axes0 )
hoo=ax1.contourf(XX,YY, Hr.T,levels=lev,cmap=cmap)
foo=ax1.plot( [-3,4],[160,0] ,color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(0,160)

ax2 = fig.add_axes( Axes1 )
ax2.plot( XX, np.sum( Hr.T , axis=0) )
ax2.set_xlim(xr0,xr1)
#ax1.set_ylim(0,160)


ax3 = fig.add_axes( Axes2 )
ax3.plot( np.sum( Hr.T , axis=1),YY )
#ax1.set_xlim(-3,3)
ax3.set_xlim(0.01,1)
ax3.set_xscale('log')
ax3.set_ylim(0,160)


cax=fig.add_axes( [ .2,-.2,.5,.02 ] )
fig.colorbar( hoo, cax=cax, orientation='horizontal' )
#plt.contourf(XX,YY, H0.T,levels=lev,cmap=cmap)
#plt.colorbar()

In [ ]:
plt.scatter( x0, y0 )
plt.scatter( x, y )
plt.scatter( xo, yo )


In [ ]:
plt.contour(Hr.T,levels=lev,cmap=cmap,linewidths=4)

In [ ]:
plt.plot(Hr[6,1:]/Hr[6,1])
plt.plot(H0[6,1:]/H0[6,1])

In [ ]:
lev=np.array( [0.001, 0.002,0.005,0.01,0.02,0.03,0.05,0.07,0.1] )
lev=np.array( [0.002,0.005,0.01,0.02,0.03,0.05,0.07,0.1] )
print(lev)

cmap=Pu.step_cmap(levels=lev,cmapN='gist_ncar')
nxplo,nyplo=3,1

fig = plt.figure(figsize=(20, 5))

xr0,xr1= -3,2 #-4,2.7 #-3,3.7
b=1
#plt.plot( BACE_sst1[b,:,:].flatten() )

Axes0 = Pu.axes_def(n=1,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
#hoo=ax1.contour(XX,YY, H0.T,levels=lev,colors='blue',linewidths=4)
#hoo2=ax1.contour(XX,YY, Hr.T,levels=lev,colors='red',linewidths=4)
ska=ax1.scatter( x0, y0 ,s=5, color='blue')
ska2=ax1.scatter( x, y ,s=5, color='red')
foo=ax1.plot( [-5,2],[160,0] ,color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(0,160)


"""
###################
Axes0 = Pu.axes_def(n=2,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes0 )
hoo=ax1.contour(XX,YY, H0.T,levels=lev,cmap=cmap,linewidths=4)
ska=ax1.scatter( x0, y0 ,s=10, color='black')
foo=ax1.plot( [-5,2],[160,0] ,color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(0,160)


Axes0 = Pu.axes_def(n=3,nxplo=nxplo,nyplo=nyplo) 
ax1 = fig.add_axes( Axes0 )
hoo=ax1.contour(XX,YY, Hr.T,levels=lev,cmap=cmap,linewidths=4)
foo=ax1.plot( [-5,2],[160,0] ,color='black')
ska=ax1.scatter( x,  y ,s=10, color='black')
#plt.colorbar(hoo, location='bottom')
ax1.set_xlim(xr0,xr1)
ax1.set_ylim(0,160)


cax=fig.add_axes( [ .2,-.2,.5,.02 ] )
fig.colorbar( hoo, cax=cax, orientation='horizontal' )
#plt.contourf(XX,YY, H0.T,levels=lev,cmap=cmap)
#plt.colorbar()
"""

In [ ]:
#######################################################
#######################################################


In [ ]:
boo=np.sum(BACE_sst7,axis=2)
yoo=np.stack( (BACE_sst7[1,:,mon0-1:mon1], BACE_sst7[1,:,mon0-1:mon1] )).flatten()

In [ ]:
plt.plot(boo[1,:])
print(np.average( boo[1,:]) )
print(np.average( 7*yoo ))